# 姿态估计

姿态估计（pose estimation）：检测图像或视频帧中的特定关键点，以跟踪运动或姿势。

姿态估计是一项任务，它涉及识别图像中特定点的位置，这些点通常被称为关键点。这些关键点可以代表物体的各个部分，如关节、地标或其他独特特征。关键点的位置通常以一组二维 `[x, y]` 或三维 `[x, y, visible]` 坐标来表示。

姿态估计模型的输出是一组点，这些点代表了图像中物体上的关键点，通常还包括每个点的置信度分数。当你需要识别场景中物体的具体部分及其彼此间的相对位置时，姿态估计是一个不错的选择。


```{tip}
YOLOv11 的人体姿态估计模型使用了 `-pose` 后缀，例如 `yolo11n-pose.pt`。这些模型是在 [COCO keypoints](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco-pose.yaml) 数据集上训练的，适用于多种姿态估计任务。

在默认的 YOLOv11 姿态模型中，共有 17 个关键点，每个关键点代表人体的不同部位。下面是每个索引对应的身体关节：

0: 鼻子
1: 左眼
2: 右眼
3: 左耳
4: 右耳
5: 左肩
6: 右肩
7: 左肘
8: 右肘
9: 左腕
10: 右腕
11: 左髋
12: 右髋
13: 左膝
14: 右膝
15: 左踝
16: 右踝
```

## 模型

这里展示了YOLOv11预训练的姿态估计模型。检测、分割和姿态估计模型是在[COCO](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco.yaml)数据集上预训练的，而分类模型则是在[ImageNet](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/ImageNet.yaml)数据集上预训练的。

模型会在首次使用时自动从最新的Ultralytics [发布](https://github.com/ultralytics/assets/releases)中下载。
| 模型                                                                                          | 尺寸<br><sup>(像素) | mAP<sup>pose<br>50-95 | mAP<sup>pose<br>50 | CPU ONNX速度<br>(毫秒) | T4 TensorRT10速度<br>(毫秒) | 参数<br><sup>(百万) | FLOPs<br><sup>(十亿次) |
| ---------------------------------------------------------------------------------------------- | --------------------- | --------------------- | ------------------ | ------------------------------ | ----------------------------------- | ------------------ | ----------------- |
| [YOLOv11n-pose](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-pose.pt) | 640                   | 50.0                  | 81.0               | 52.4 ± 0.5                     | 1.7 ± 0.0                           | 2.9                | 7.6               |
| [YOLOv11s-pose](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11s-pose.pt) | 640                   | 58.9                  | 86.3               | 90.5 ± 0.6                     | 2.6 ± 0.0                           | 9.9                | 23.2              |
| [YOLOv11m-pose](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m-pose.pt) | 640                   | 64.9                  | 89.4               | 187.3 ± 0.8                    | 4.9 ± 0.1                           | 20.9               | 71.7              |
| [YOLOv11l-pose](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11l-pose.pt) | 640                   | 66.1                  | 89.9               | 247.7 ± 1.1                    | 6.4 ± 0.1                           | 26.2               | 90.7              |
| [YOLOv11x-pose](https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11x-pose.pt) | 640                   | 69.5                  | 91.1               | 488.0 ± 13.9                   | 12.1 ± 0.2                          | 58.8               | 203.3             |

- **mAP<sup>val</sup>** 值是基于[COCO Keypoints val2017](https://cocodataset.org/)数据集上的单模型单尺度计算得出的。<br>可以通过`yolo val pose data=coco-pose.yaml device=0`重现。
- **速度**是使用Amazon EC2 P4d实例在COCO验证图像上的平均速度。<br>可以通过`yolo val pose data=coco-pose.yaml batch=1 device=0|cpu`重现。

## 训练

在COCO8-pose数据集上训练 YOLOv11-pose 模型。
```python
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.yaml")  # build a new model from YAML
model = YOLO("yolo11n-pose.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n-pose.yaml").load("yolo11n-pose.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="coco8-pose.yaml", epochs=100, imgsz=640)
```

## 验证

在COCO8-pose数据集上验证已训练的YOLOv11n-pose模型的准确性。由于`model`保留了其训练数据和参数作为模型属性，因此不需要任何额外的参数。

```python
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
model = YOLO("path/to/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category
```

## 预测

使用训练好的YOLOv11n-pose模型对图像进行预测。
```python
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
model = YOLO("path/to/best.pt")  # load a custom model

# Predict with the model
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
```